In [1]:
import pandas as pd
import endpoints
from datetime import datetime, timedelta
# from config import SITE_EMAIL, SITE_PASSWORD
import time
import numpy as np

In [2]:
import json

In [3]:
from urllib.parse import quote

In [4]:
today = datetime.today()
yesterday = today - timedelta(days=1)
today_str = today.strftime('%Y-%m-%d')
yesterday_str = yesterday.strftime('%Y-%m-%d')

In [6]:
cookies_dict = endpoints.get_cookies_local(EMAIL, PASSWORD)
session = endpoints.get_session()

In [ ]:

data = endpoints.get_l1_l2(yesterday_str, today_str, session, cookies_dict)

df = pd.json_normalize(data['data'])
df.rename({'id': 'l1_id', 'value': 'l1_name'}, axis=1, inplace=True)
df = df[['l1_id', 'l1_name', 'data']]\
    .explode('data')\
    .reset_index(drop=True)
df = df.drop(0)\
    .reset_index(drop=True)
df_expanded = pd.json_normalize(df['data'])
df = pd.concat([df[['l1_id', 'l1_name']], df_expanded], axis=1)\
    .rename({'id': 'l2_id', 'value': 'l2_name'}, axis=1)
df = df[['l1_id', 'l1_name', 'l2_id', 'l2_name']]

In [7]:
temp_dfs = []
for l2_id in df['l2_id']:
    data = endpoints.get_l3(yesterday_str, l2_id, today_str, session, cookies_dict)
    df_temp = pd.json_normalize(data['data'])
    df_temp['l2_id'] = l2_id
    temp_dfs.append(df_temp)
    time.sleep(1)

if temp_dfs:
    df_l3 = pd.concat(temp_dfs, ignore_index=True)
    df_l3 = df_l3.rename(columns={'id': 'l3_id', 'name': 'l3_name'})\
        .drop(['url', 'path', 'isFavorite'], axis=1)
else:
    df_l3 = pd.DataFrame()

In [7]:
def get_wb_totals():
    # Параметры запроса
    category_id = "12413804913918531246"
    check_date = "2025-09-10"
    period_days = "30"

    # Формируем JSON query
    query_params = {
        "start": 0,
        "length": 0,
        "orderBy": "ordersSum",
        "orderDirection": "desc",
        "checkDate": check_date,
        "periodDays": period_days,
        "trendType": "day",
        "filters": {}
    }

    # Кодируем JSON для URL
    encoded_query = quote(json.dumps(query_params))

    # Формируем timestamp
    timestamp = datetime.now().strftime("%Y-%m-%d_%H-0")

    # Собираем итоговый URL
    url = f"https://eggheads.solutions/analytics/wbCategory/getBrandsList/{category_id}.json?query={encoded_query}&dns-cache={timestamp}"

    return url

In [8]:
l3_id = '12413804913918531246'

In [9]:
import requests

In [10]:
response = requests.get(get_wb_totals(), cookies=cookies_dict)

In [13]:
data = response.json()

,ordersSum,orders,dt
0,25652,14,2025-08-12 00:00:00+00:00
1,25678,17,2025-08-13 00:00:00+00:00
2,39771,22,2025-08-14 00:00:00+00:00
3,19624,13,2025-08-15 00:00:00+00:00
4,43742,27,2025-08-16 00:00:00+00:00
5,28443,14,2025-08-17 00:00:00+00:00
6,31479,17,2025-08-18 00:00:00+00:00
7,32790,15,2025-08-19 00:00:00+00:00
8,21538,13,2025-08-20 00:00:00+00:00
9,27910,12,2025-08-21 00:00:00+00:00


In [22]:
df

,date,ordersSum,orders,dt
0,2025-08-12T00:00:00+00:00,25652,14,2025-08-12 00:00:00+00:00
1,2025-08-13T00:00:00+00:00,25678,17,2025-08-13 00:00:00+00:00
2,2025-08-14T00:00:00+00:00,39771,22,2025-08-14 00:00:00+00:00
3,2025-08-15T00:00:00+00:00,19624,13,2025-08-15 00:00:00+00:00
4,2025-08-16T00:00:00+00:00,43742,27,2025-08-16 00:00:00+00:00
5,2025-08-17T00:00:00+00:00,28443,14,2025-08-17 00:00:00+00:00
6,2025-08-18T00:00:00+00:00,31479,17,2025-08-18 00:00:00+00:00
7,2025-08-19T00:00:00+00:00,32790,15,2025-08-19 00:00:00+00:00
8,2025-08-20T00:00:00+00:00,21538,13,2025-08-20 00:00:00+00:00
9,2025-08-21T00:00:00+00:00,27910,12,2025-08-21 00:00:00+00:00


In [16]:
df = df.explode('trend')

In [ ]:
df = df[['']]

,brandId,lastRemains,reviewsCount,name,isFavorite,orders,ordersSum,loosesPercent,avgSum,loosesSum,trend,productsCount,newProductsCount,orderedCardsSum,sharePercent
0,None,None,None,Автотовары > OFFroad > Антипробуксовочное прис...,False,755,1523102,39.77,2017,605759,"{'date': '2025-08-12T00:00:00+00:00', 'ordersS...",None,None,0,0
0,None,None,None,Автотовары > OFFroad > Антипробуксовочное прис...,False,755,1523102,39.77,2017,605759,"{'date': '2025-08-13T00:00:00+00:00', 'ordersS...",None,None,0,0
0,None,None,None,Автотовары > OFFroad > Антипробуксовочное прис...,False,755,1523102,39.77,2017,605759,"{'date': '2025-08-14T00:00:00+00:00', 'ordersS...",None,None,0,0
0,None,None,None,Автотовары > OFFroad > Антипробуксовочное прис...,False,755,1523102,39.77,2017,605759,"{'date': '2025-08-15T00:00:00+00:00', 'ordersS...",None,None,0,0
0,None,None,None,Автотовары > OFFroad > Антипробуксовочное прис...,False,755,1523102,39.77,2017,605759,"{'date': '2025-08-16T00:00:00+00:00', 'ordersS...",None,None,0,0
0,None,None,None,Автотовары > OFFroad > Антипробуксовочное прис...,False,755,1523102,39.77,2017,605759,"{'date': '2025-08-17T00:00:00+00:00', 'ordersS...",None,None,0,0
0,None,None,None,Автотовары > OFFroad > Антипробуксовочное прис...,False,755,1523102,39.77,2017,605759,"{'date': '2025-08-18T00:00:00+00:00', 'ordersS...",None,None,0,0
0,None,None,None,Автотовары > OFFroad > Антипробуксовочное прис...,False,755,1523102,39.77,2017,605759,"{'date': '2025-08-19T00:00:00+00:00', 'ordersS...",None,None,0,0
0,None,None,None,Автотовары > OFFroad > Антипробуксовочное прис...,False,755,1523102,39.77,2017,605759,"{'date': '2025-08-20T00:00:00+00:00', 'ordersS...",None,None,0,0
0,None,None,None,Автотовары > OFFroad > Антипробуксовочное прис...,False,755,1523102,39.77,2017,605759,"{'date': '2025-08-21T00:00:00+00:00', 'ordersS...",None,None,0,0


In [3]:
pd.set_option('display.max_rows', None)

In [4]:
df = pd.read_csv('l1_l2_data.csv')

In [31]:
# Заменяем строки с кавычками на NaN
df['l2_id'] = df['l2_id'].replace('', pd.NA, regex=False)

# Заполняем NaN значениями из l1_id
df['l2_id'] = df['l2_id'].fillna(df['l1_id'])